In [1]:
# 导入必须的包
from langchain_community.document_loaders import UnstructuredExcelLoader, Docx2txtLoader, PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import  ChatOpenAI
# 导入聊天所需的模块
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
import os
from dotenv import load_dotenv

load_dotenv()

# 定义chatdoc
class ChatDoc():
    def __init__(self):
        self.doc = None
        self.splitText = []  # 分割后的文本
        self.template = [
            ("system", "你是一个处理文档的秘书,你从不说自己是一个大模型或者AI助手,你会根据下面提供的上下文内容来继续回答问题.\n 上下文内容\n {context} \n"),
            ("human", "你好！"),
            ("ai", "你好"),
            ("human", "{question}"),
        ]
        self.prompt = ChatPromptTemplate.from_messages(self.template)

    def getFile(self):
        doc = self.doc
        loaders = {
            "docx": Docx2txtLoader,
            "pdf": PyPDFLoader,
            "xlsx": UnstructuredExcelLoader,
        }
        file_extension = doc.split(".")[-1]
        loader_class = loaders.get(file_extension)
        if loader_class:
            try:
                loader = loader_class(doc)
                text = loader.load()
                return text
            except Exception as e:
                print(f"Error loading {file_extension} files:{e}")
                return None
        else:
            print(f"Unsupported file extension: {file_extension}")
            return None

    # 处理文档的函数
    def splitSentences(self):
        full_text = self.getFile()  # 获取文档内容
        if full_text is not None:
            # 对文档进行分割
            text_split = CharacterTextSplitter(
                chunk_size=150,
                chunk_overlap=20,
            )
            texts = text_split.split_documents(full_text)
            self.splitText = texts

    # 向量化与向量存储
    def embeddingAndVectorDB(self):
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        db = Chroma.from_documents(
            documents=self.splitText,
            embedding=embeddings,
            persist_directory="./chroma_db"  # v1.0 推荐指定持久化目录
        )
        return db

    # 提问并找到相关的文本块
    def askAndFindFiles(self, question):
        db = self.embeddingAndVectorDB()
        # retriever = db.as_retriever(search_type="mmr")
        retriever = db.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={"score_threshold": 0.5, "k": 1}
        )
        return retriever.invoke(question)  # v1.0 使用 invoke 方法

    # 用自然语言和文档聊天
    def chatWithDoc(self, question):
        _content = ""
        context = self.askAndFindFiles(question)
        for i in context:
            _content += i.page_content

        messages = self.prompt.format_messages(context=_content, question=question)
        chat = ChatOpenAI(
            model="gpt-3.5-turbo:free",
            base_url=os.getenv("OPENAI_BASE_URL"),
            api_key=os.getenv("OPENAI_API_KEY"),
            temperature=0 # 温度是指的模型输出结果的随机性。分数越高，输出结果越随机。
        )
        return chat.invoke(messages)


chat_doc = ChatDoc()
chat_doc.doc = "resource/example/fake.docx"
chat_doc.splitSentences()
result = chat_doc.chatWithDoc("公司注册地址是哪里？")
print(result.content)

公司注册地址是江苏省南京市雨花台区软件大道101号。
